## 神经脆弱性矩阵计算
### 1.分窗

In [ ]:
def get_window_signals(full_signal, window_size, repeat_rate = 0.5):
    """
    滑动分窗
    输入:
    full_signal，形状 n_channel, n_time
    window_size, 窗大小
    repeat_rate, 滑窗的重复率
    输出：
    window_signals, 列表，每一个元素是一个信号窗（形状为n_channel, window_size）
    
    """
    window_signals = []
    length = full_signal.shape[1]
    n_window = length // window_size
    step = int(window_size * repeat_rate)
    _c = 0
    while _c <= length - window_size:
        window_signals.append(full_signal[:,_c: _c + window_size])
        _c += step    
    return window_signals

### 2.构造线性方程组

In [ ]:
def get_linear_equation(signal):
    """
    论文中求解线性系统x(t+1)=Ax(t)时，将这个问题转化为了求解线性方程组的最小二乘解
    转化后的线性方程组为 H*X=b
    此函数即根据输入的信号，求得H，X和b
    输入：
    signal 形状为 (n_window, n_channel)
    输出：
    H 形状为 (n_window-1)*n_channel, n_channel*n_channel
    X 形状为 n_channel*n_channel
    b 形状为 (n_window-1)*n_channel)
    """
    
    b = []
    H = []
    X = []
    n_window, n_channel = signal.shape
    
    # get b
    for n_w in range(1, n_window):
        for n_c in range(n_channel):
            b.append(signal[n_w][n_c])
    
    # get X
    X = np.random.random([n_channel*n_channel])
    
    # get H
    for n_w in range(n_window-1):
        this_block = []
        for n_t in range(n_channel):
            this_row = np.zeros((n_channel*n_channel))
            for i in range(n_t*n_channel, (n_t+1)*n_channel):
                _this_channel = i % n_channel
                this_row[i] = signal[n_w][_this_channel]
            this_block.append(this_row)
        this_block = np.array(this_block)
        H.append(this_block)
    
    b = np.array(b)
    X = np.array(X)
    H = np.concatenate(H, 0)
    
    return H, X, b

### 3.求最小二乘解

In [ ]:
def solve_equation_by_LeastSquares(H, X, b):
    """
    求线性方程组H * X = b的最小二乘解，
    公式:
    x = (A^T * A)^(-1) * A^T * b
    """
    x = np.matmul(H.T, H)
    x = np.linalg.inv(x)
    x = np.matmul(x, H.T)
    x = np.matmul(x, b)
    return x

### 4. 求状态转移矩阵A

In [ ]:
def get_A(X):
    n = int(np.sqrt(X.shape[0]))
    return np.reshape(X, (n,n))

### 5.求扰动矩阵

In [ ]:
def get_B(A, k, sigma, omega):
    def get_elementary(k, n_channel=2):
        """
        获取n_channel维空间的第k个单位向量
        输出：
        eletmentary, 形状为 n_channel
        """
        ret = np.zeros((n_channel, 1))
        ret[k] = 1
        return ret
    e = get_elementary(k)
    I = np.identity(A.shape[0])
    
    A_modified = A - (sigma + omega * 1j) * I
    A_modified = np.linalg.inv(A_modified)
    A_modified = A_modified.T
    B_complecated = np.matmul(e.T, A_modified)
    
    B_real = B_complecated.real
    B_imag = B_complecated.imag
    
    B = np.concatenate([B_real, B_imag], 0)
    
    b = np.array([0,-1]).T
    return B, b, e


def get_perturbation(B, b, e):
    """
    get perturbation following Theorem 1 in paper.
    输入 
    B 形状 (n_channel, n_channel)
    b 形状 (1, n_channel)
    e 形状 (n_channel, n_channel)
    
    """
    ret = np.matmul(B, B.T)
    ret = np.linalg.inv(ret)
    ret = np.matmul(B.T, ret)
    ret = np.matmul(ret, b)
    ret = np.matmul(ret, e.T)
    return ret

### 6.求神经脆弱性矩阵

In [ ]:
def get_neural_fragility(full_signal, sigma, omega):
    """
    计算神经脆弱性
    1.分窗，获取若干个window_signal
    2.对每一个window_signal，计算扰动向量
    3.获取扰动矩阵
    输入：
    full_signal， 形状 n_channel, n_time
    输出：
    脆弱性矩阵 形状 n_channel, n_window_signals
    """
    # 分窗
    window_signals = get_window_signals(full_signal)
    
    # 对每一个信号窗，计算脆弱性向量
    neural_fragility_matrix = []
    for window_signal in window_signals:
        # 构造线性方程组
        H,X,b = get_linear_equation(window_signal)
        
        # 求线性方程组最小二乘解
        X = solve_equation_by_LeastSquares(H, X, b)
        
        # 获取A
        A = get_A(X)
        
        # 计算B
        neural_fragility_vector = []
        for k in range(n_channel):
            B, b, e = get_B(A, k, sigma, omega)
            # 计算神经脆弱性向量
            perturbation_matrix = get_perturbation(B, b, e)
            perturbation_matrix_norm = np.sqrt(np.sum(perturbation_matrix*perturbation_matrix))
            neural_fragility_vector.append(perturbation_matrix_norm)
            
        neural_fragility_matrix.append(neural_fragility_vector)
    
    # 转换为论文中的矩阵格式：每一列是一个窗信号的脆弱性向量
    neural_fragility_matrix = np.array(neural_fragility_matrix)
    neural_fragility_matrix = neural_fragility_matrix.T
    return neural_fragility_matrix